In [1]:
import pandas as pd
import geopandas as gp
import numpy as np 
import xlrd
import os
import fiona
from statistics import mean, median
import string
import math
import re

from shapely.geometry import Polygon
from shapely.ops import cascaded_union

# VEST IA 2020

## VEST Documentation

### Sources

Election results from Iowa Secretary of State (https://sos.iowa.gov/elections/results/index.html)<br>
Precinct shapefile primarily from IA Secretary of State (https://sos.iowa.gov/shapefiles/)

### Shapefiles

The following counties instead use shapefiles sourced from the respective county governments: Dallas, Dubuque, Johnson, Linn, Polk, Pottawattamie, Scott, Story. The following counties were aligned with updated municipal boundaries from the U.S. Census Bureau's 2020 Redistricting Data Program: Marion, Muscatine, Woodbury. Mitchell County was edited to match the revised 2020 precinct map from the county.

The following precincts were merged to match the 2020 general election results.

Appanoose: Udell/Union<br>
Black Hawk: Cedar Falls W2P2/Cedar Falls Twp<br>
Cass: Atlantic 1/BR-GV-WS-PY, Atlantic 2/3, 4/5, BN-FR/GN-LN<br>
Davis: Bloomfield 1/2/3<br>
Des Moines: Burlington 1/Tama, Burlington 8/Concordia<br>
Fremont: Hamburg/Washington, Farragut/Shenandoah1<br>
Henry: Mt Pleasant 1/2/3/4<br>
Howard: Cresco 1/2/3<br>
Lee: Fort Madison 4A/4B, Keokuk 2A/2B, FCM/Harrison<br>
Mitchell: Precinct 1/12, 3/10/11, 5/6, 7/8/9<br>
Polk: Grimes 2/Urbandale 12<br>
Tama: Toledo 1/2/3<br>

### Races

G20PRERTRU - Donald J. Trump (Republican Party)<br>
G20PREDBID - Joseph R. Biden (Democratic Party)<br>
G20PRELJOR - Jo Jorgensen (Libertarian Party)<br>
G20PREGHAW - Howie Hawkins (Green Party)<br>
G20PRECBLA - Don Blakenship (Constitution Party)<br>
G20PREOFUE - Roque Rocky De La Fuente (Alliance Party)<br>
G20PREOKIN - Ricki Sue King (Genealogy Know Your Family History Party)<br>
G20PREIWES - Kanye West (Independent)<br>
G20PREIPIE - Brock Pierce (Independent)<br>
G20PREOWRI - Write-in Votes<br>
<br>
G20USSRERN - Joni Ernst (Republican Party)<br>
G20USSDGRE - Theresa Greenfield (Democratic Party)<br>
G20USSLSTE - Rick Stewart (Libertarian Party)<br>
G20USSOHER - Suzanne Herzog (Independent)<br>
G20USSOWRI - Write-in Votes<br>

## Load in VEST file

In [ ]:
vest_ia